In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

adult = pd.read_csv("data/adult_training.csv",
                      delimiter=",",
                      skipinitialspace=True,
                        #nrows=10000,
                      dtype=None)

adult_test = pd.read_csv("data/adult_training.csv",
                      delimiter=",",
                      skipinitialspace=True,
                        #nrows=10000,
                      dtype=None)

from sklearn.model_selection import train_test_split
from __future__ import division
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.ensemble import BaggingClassifier

from joblib import dump, load
from sklearn.model_selection import cross_val_score
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
def adult_preprocess_balanced(adult):
    """Takes in an adult income pandas dataframe, removes '?', 
    expands categorical data returns X and Y arrays"""
    
    # remove rows with '?'s
    adult = adult[(adult != '?').all(1)]
    
    # convert categorical data into one-hot
    adult_one_hot = pd.get_dummies(adult)
    
    adult_over_50k = adult_one_hot[adult_one_hot['income_>50K'] == 1].sample(n=7500, random_state=0)
    adult_under_50k = adult_one_hot[adult_one_hot['income_>50K'] == 0].sample(n=7500, random_state=0)
    
    frames = [adult_over_50k, adult_under_50k]
    
    adult_clean = pd.concat(frames)
    adult_clean = adult_clean.sample(frac=1)
    
    # split into inputs and targets
    X = adult_clean.iloc[:,0:-2].values
    Y = adult_clean.loc[:,'income_>50K'].values
    
    return X, Y

In [4]:
def adult_preprocess_unbalanced(adult):
    """Takes in an adult income pandas dataframe, removes '?', 
    expands categorical data returns X and Y arrays"""
    
    # remove rows with '?'s
    adult = adult[(adult != '?').all(1)]
    
    # convert categorical data into one-hot
    adult_one_hot = pd.get_dummies(adult)

    # split into inputs and targets
    X = adult_one_hot.iloc[:,0:-2].values
    Y = adult_one_hot.loc[:,'income_>50K'].values
    
    return X, Y

In [5]:
X, Y = adult_preprocess_balanced(adult)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
X = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

X_test, Y_test = adult_preprocess_balanced(adult_test)
X_test = scaler.fit_transform(X_test)

In [6]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(104, input_dim=104, kernel_initializer='normal', activation='relu'))
    model.add(Dense (60,kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
clf = create_baseline()
print X_train.shape
clf.fit(X_train,Y_train,batch_size=1,epochs=5)

(12000, 104)
Epoch 1/5
12000/12000 [==============================] - 24s 2ms/step - loss: 0.4314 - acc: 0.7957
Epoch 2/5
12000/12000 [==============================] - 24s 2ms/step - loss: 0.3970 - acc: 0.8151
Epoch 3/5
12000/12000 [==============================] - 24s 2ms/step - loss: 0.3870 - acc: 0.8238
Epoch 4/5
12000/12000 [==============================] - 23s 2ms/step - loss: 0.3797 - acc: 0.8247
Epoch 5/5
12000/12000 [==============================] - 24s 2ms/step - loss: 0.3763 - acc: 0.8281
